In [0]:
from datetime import datetime
import json
from pyspark.sql.functions import to_date, col, current_date
import pandas as pd


In [0]:
%sql
-- show catalogs

In [0]:
%sql
use catalog job_marketplace

In [0]:
%sql
-- show schemas

In [0]:
df = spark.sql('select * from job_marketplace.silver_layer.clean_jobs')
# display(df)

In [0]:
# %pip install openpyxl

In [0]:
# silver layer - cleaned_data
def export_to_excel(filename,df):
    path = f"/Workspace/Users/varunyadav032003@gmail.com/databricks_apps/hello-world_2025_07_11-09_57/streamlit-hello-world-app/data/{filename}"
    pdf = df.toPandas()
    pdf.to_excel(path, index=False, engine='openpyxl')
    print(f"File saved to: {path}")


In [0]:
# Get current timestamp in ISO format
last_run_ts  = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
record_count = df.count()
new_jobs_today = df.filter(
    to_date(col("posted_at")) == current_date()
).count()

print(f"New jobs today: {new_jobs_today}")
print(f"Last successful run: {last_run_ts}")
print(f"Record count: {record_count}")

In [0]:
# Create dictionary
status = {
          "last_successful_run": last_run_ts ,
          "record_count": record_count,
          "new_jobs_today": new_jobs_today
}
status_json_df = spark.createDataFrame([status])

display(status_json_df)

In [0]:

def export_status_to_json(spark_df, filename):
    path = "/Workspace/Users/varunyadav032003@gmail.com/databricks_apps/hello-world_2025_07_11-09_57/streamlit-hello-world-app/data/"
    
    # Convert to Pandas
    pdf = spark_df.toPandas()

    # Export as JSON
    pdf.to_json(f"{path}{filename}", orient="records", lines=False, indent=4)
    
    print(f"✅ JSON file saved to: {path}{filename}")


In [0]:
df_job_posting_trend = spark.sql('select * from job_marketplace.gold_layer.kpi_job_posting_trend')
df_jobs_by_location_city = spark.sql('select * from job_marketplace.gold_layer.kpi_jobs_by_location_city')
df_remote_vs_onsite = spark.sql('select * from job_marketplace.gold_layer.kpi_remote_vs_onsite')
df_top_companies = spark.sql('select * from job_marketplace.gold_layer.kpi_top_companies')

In [0]:
export_to_excel('jobs_posting_trend.xlsx',df_job_posting_trend)
export_to_excel('jobs_by_location_city.xlsx',df_jobs_by_location_city)
export_to_excel('remote_vs_onsite.xlsx',df_remote_vs_onsite)
export_to_excel('top_companies.xlsx',df_top_companies)

In [0]:
# export_to_excel("pipeline_status.json",df)
export_status_to_json(status_json_df,'pipeline_status.json')